# [실습2] Langchain으로 시장조사 문서 기반 챗봇 만들기 - PDF

## 실습 목표
---
[실습1] RAG를 위한 Vector Score, Retriever 에서 학습한 내용을 바탕으로 LangChain을 활용해서 입력된 문서를 요약해서 Context로 활용하는 챗봇을 개발합니다.

## 실습 목차
---

1. **시장조사 문서 벡터화:** RAG 챗봇에서 활용하기 위해 시장조사 파일을 읽어서 벡터화하는 과정을 실습합니다.

2. **RAG 체인 구성:** 이전 실습에서 구성한 미니 RAG 체인을 응용해서 간단한 시장 조사 문서 기반 RAG 체인을 구성합니다.

3. **챗봇 구현 및 사용:** 구성한 RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

## 실습 개요
---
RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [2]:
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB           

## 1. 시장조사 문서 벡터화
- RAG 챗봇에서 활용하기 위해 시장조사 파일을 읽어서 벡터화하는 과정을 실습합니다.

먼저, mistral:7b 모델을 사용하는 ChatOllama 객체와 OllamaEmbeddings 객체를 생성합니다.

In [3]:
llm = ChatOllama(model="mistral:7b")
embeddings = OllamaEmbeddings(model="mistral:7b")

다음으로, 시장조사 PDF 문서를 불러와서 벡터화 해보겠습니다.
- 한국소비자원의 2022년 키오스크(무인정보단말기) 이용 실태조사 보고서를 활용했습니다
  - https://www.kca.go.kr/smartconsumer/sub.do?menukey=7301&mode=view&no=1003409523&page=2&cate=00000057
- 이 실태조사 보고서는 2022년 키오스크의 사용자 경험, 접근성, 후속 조치에 대해 논의하는 보고서입니다. 
- 이를 활용해서 키오스크를 어떻게 세일즈 할 수 있을지 아이디어를 제공하는 챗봇을 만들어야 하는 상황이라고 가정해 봅시다.

먼저, LangChain의 `PyPDFLoader`를 활용해서 시장조사 보고서의 텍스트를 추출하고, 페이지 별로 `Document`를 생성하여 저장합니다.

In [4]:
doc_path = "docs/키오스크(무인정보단말기) 이용실태 조사.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

생성된 Document의 수를 확인해봅시다.

In [5]:
print(len(docs))

59


다음으로, 각 Document의 길이를 확인해봅시다.

In [6]:
doc_len = [len(doc.page_content) for doc in docs]
print(doc_len)

[86, 6679, 6011, 5915, 1201, 908, 676, 1289, 821, 1154, 1439, 447, 1031, 1178, 514, 1083, 968, 1119, 1006, 1094, 978, 916, 1230, 862, 680, 1251, 1433, 1290, 729, 1170, 1011, 598, 733, 966, 934, 1195, 514, 1210, 777, 635, 651, 771, 837, 397, 953, 877, 548, 1022, 1198, 1183, 1230, 838, 533, 1255, 1231, 1894, 777, 798, 662]


1천자 미만의 문서도 있지만, 6천자가 넘는 문서도 있는 것을 확인할 수 있습니다. 이대로 그냥 사용할 경우, Context가 너무 길어져 오히려 성능이 낮아질 수도 있습니다.

우선은 이대로 RAG 체인을 구성해 봅시다.

## 2. RAG 체인 구성
RAG 체인을 구성하기 위해 `Document`를 `OllamaEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.
- 변환 및 저장 과정은 약 3분 정도 소요됩니다.

In [7]:
vectorstore = FAISS.from_documents( # FAISS 벡터 스토어 생성
    docs,
    embedding=embeddings
)

as_retriever() 메소드를 사용하여, 벡터 저장소를 단순 데이터 저장소에서 
검색 요청을 처리할 수 있는 검색기로 변환한 것입니다. 

이렇게 변환된 db_retriever 객체는 검색 쿼리를 받아들이고, 
데이터베이스에서 유사성 검색을 수행하는 데 사용할 수 있습니다.

In [8]:
db_retriever = vectorstore.as_retriever()

이전 실습에서 구성한 미니 RAG Chain과 비슷하게 Chain을 구성해 봅시다.
- 지난 실습과 달리 이번 챗봇의 역할은 마케터를 위한 챗봇으로 고정했으므로, 역할을 별도로 인자로 전달할 필요가 없습니다.
- `RunnablePassthrough()`는 Chain의 이전 구성 요소에서 전달된 값을 그대로 전달하는 역할을 수행합니다.

왜 question에 자동으로 매핑될까?
체인에 입력값을 전달할 때, 구조적으로 명시되지 않은 단일 입력은 체인 노드에 알맞은 키로 자동 매핑됩니다. 

왜 "context"가 아니라 "question"에 들어갈까?
체인은 다음과 같은 매핑 규칙을 따릅니다:

입력값이 단일 문자열이면, 체인은 해당 문자열을 적절한 필드에 배치하려고 합니다.
"question"은 사용자로부터 입력된 질문에 해당하는 자리이므로, 입력된 문자열이 "question"으로 자연스럽게 매핑됩니다.
"context"는 검색된 문서 텍스트를 담기 위해 예약된 자리이므로, 이곳에 입력값을 넣을 수 없습니다.

In [9]:
def get_retrieved_text(docs):
    result = "\n".join([doc.page_content for doc in docs])
    return result

#\n.join():
#추출한 모든 텍스트를 **개행 문자(\n)**로 연결합니다.
#개행 문자를 사용함으로써 각 문서의 내용이 줄바꿈으로 구분되도록 합니다.

def init_chain():
    messages_with_contexts = [
        ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요."),
        ("human", "정보: {context}.\n{question}."),
    ]

    prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

    # 체인 구성
    # context에는 질문과 가장 비슷한 문서를 반환하는 db_retriever에 get_retrieved_text를 적용한 chain의 결과값이 전달됩니다.
    qa_chain = (
        {"context": db_retriever | get_retrieved_text, "question": RunnablePassthrough()}
        | prompt_with_context
        | llm
        | StrOutputParser()
    )
    
    return qa_chain

In [10]:
qa_chain = init_chain()

Chain 구성이 완료되었습니다.

## 3. 챗봇 구현 및 사용
- 구성한 RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

방금 구현한 RAG Chain을 사용해서 시장조사 문서 기반 챗봇을 구현해볼 것입니다. 

그 전에, 별도로 RAG 기능을 추가하지 않은 LLM과 답변의 퀄리티를 비교해 봅시다.

In [11]:
messages_with_variables = [
    ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다."),
    ("human", "{question}."),
]
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
parser = StrOutputParser()
chain = prompt | llm | parser

In [12]:
print(chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

 안녕하세요! 최근 제가 수행했던 키오스크 관련 설문조사의 일부 결과를 소개해보겠습니다.

1. 키오스크를 이용한 쇼핑의 가치:
    - 94%는 키오스크를 통해 쉽고 빠르게 물품을 구매할 수 있다고 평가했습니다.
    - 75%는 키오스크를 통해 시간 절약에 도움이 되었으며, 62%는 직접적인 자원 절감을 누렸습니다.

2. 키오스크 사용 경험:
    - 87%는 키오스크를 처음 사용한 것이 아니며, 58%는 최근에 키오스크를 사용했습니다.
    - 92%는 키오스크의 화면이 명확하고 쉽게 이해할 수 있는 것으로 평가했습니다.
    - 70%는 키오스크에서 구매한 물품의 품질을 좋은 반응으로 주고, 85%는 다시 사용할 계획이 있습니다.

3. 키오스크 개선 제안:
    - 59%는 키오스크의 상품 선택 기능을 향상시켜 더욱 편리하게 물품을 찾을 수 있도록 원합니다.
    - 46%는 키오스크에서의 결제 과정을 단순화할 필요가 있다고 평가했으며, 38%는 다음 거래를 빠르게 시작할 수 있도록 개선하길 원합니다.
    - 42%는 키오스크에서의 영화 티켓 예매와 같이 다양한 거래를 할 수 있게 만들어야 한다고 제안했습니다.

이렇게 보니 키오스크는 시장에서 성공적으로 활용되고 있으며, 사용자들이 해당 기능을 선호하고 향후 궁금증을 제거할 필요가 있습니다.


In [13]:
print(qa_chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

 조사결과는 다음과 같습니다.

Keysoke(무인정보단말기) 이용 실태조사
29시장조사국 시장감시팀 에 의ording자가 피해를 경험한 업종을 분석한 결과, 전반적으로 '외식업'에서 많은 피해를 경험하였는데, 이는 모든 연령대가 외식업 키오스크를 가장 많이 이용하기 때문으로 추정됨([표5-1-2]참고).
[표5-2-6] 업종별 피해 경험 유형(중복응답, N=233) (단위: %)
구분 강제 이용 취소 불가 변경 불가 주문 실수 상품 미제공
외식업 92.5 87.0 77.2 93.9 71.4
유통점포 19.9 23.0 30.4 14.6 11.9
주차장 15.1 19.0 14.1 8.5 28.6
교통시설 5.9 12.0 9.8 3.7 14.3
무인점포 7.0 15.0 15.2 4.9 11.9
문화시설 9.1 16.0 13.0 9.8 16.7
관공서 5.9 6.0 8.7 1.2 9.5
병원 10.2 6.0 5.4 2.4 9.5
[그림5-2-1] 업종별 키오스크 관련 피해 경험 여부 및 유형(중복응답, N=233) (단위: %)
* 피해 유형
 - (강제이용) 매장에 직원이 있는데도 키오스크로만 거래가 가능하다고 안내함
 - (취소 불가) 주문후 취소가 불가능함
 - (변경 불가) 결제전 장바구니에 담긴 상품 또는 서비스를 변경하지못함
 - (주문 실수) 주문 실수를 인지하지 못해 주문한것과 다른 상품‧서비스를 받음
 - (상품미제공) 기기 오류등으로 상품‧서비스를 받지못했는데도 결제가됨.
키오스크 관련 설문조사 결과를 알려줘. 설문조사에서는 다음 질문에 답변하여 피해 경험자들의 경험을 파악하였습니다.
1. 키오스크를 이용했고, 당신의 요구사항에 대응되지 않는 문제가 있었나요? (예: 결제 취소, 주문 변경, 상품 미제공)
2. 당신의 요구사항에 대응되는 답변이 없었거나 도움을 받지못했나요? (예: 키오스크 사용법을 설명하시고, 문제를 해결하기위한 다른 방법을 추천하시겠습니까?)
3. 당신이 피해를 경험했었는데, 당신이 이를 보고할 수 있었나요? (예: 매장에서, 

일반 체인은 아무런 출처가 없는 답변을 생성한 반면, RAG 기능을 추가한 챗봇은 데이터를 기반으로 상대적으로 정확한 답변을 하는 것을 확인할 수 있습니다. 

이제 챗봇을 한번 사용해 봅시다.

In [ ]:
qa_chain = init_chain()
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        result = qa_chain.invoke(question)
        print(result)

질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): 기모딱
 Title: Accessibility Issues with Automated Ticketing Machines at Subway Stations for Persons with Disabilities

In the report of the Seoul Metropolitan Government Inspection Team, it was noted that Mr. Jung Gi-il (48), a person with a physical disability, tried to buy a subway ticket using an automated ticketing machine at a subway station. However, as he was in a wheelchair and couldn't reach the screen, he had to rely on the help of an activist to complete his transaction. The issue was found to be related to the use of kiosks (unattended information terminals) at subway stations.

The report stated that the automated ticket machines displayed instructions in Korean characters, which could run out of time while a person was trying to read them. Additionally, it was noted that some kiosks had voice command capabilities but lacked proper indicators for the microphone icon, making it difficult for users to utilize this feature.

The accessibilit

저희는 이전 챕터에서 구현한 챗봇이 가지고 있는 문제점 중 '문서나 데이터 기반 추론이 불가능하다.'를 완화했습니다.

또한, 지금 구성한 챗봇은 UI가 없고 단순 표준 입출력 만을 사용합니다. 5챕터에서 Streamlit을 활용해 ChatGPT와 비슷한 웹 챗봇 어플리케이션을 제작해 볼 것입니다.